## Risk Mediation for Heart Failure for BMI to FPG and SBP
This notebook is designed to calculate the mediation factors for heart failure for FPG and SBP. LDL-C does not directly affect HF in our model, and so does not have a mediation factor here. We also test if we need to include the risk mediation of BMI on FPG based on the results of a trial and the observed effect on FPG.

### First, we will load in the relative risk values from GBD


In [1]:
from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws
import numpy as np
import pandas as pd

In [2]:
ihd_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best',
                    decomp_step='step4')

ihd_bmi = ihd_bmi.loc[ihd_bmi.cause_id==493].reset_index()
ihd_bmi = ihd_bmi[['age_group_id','sex_id','draw_0']]
ihd_bmi['age_start'] = np.where(ihd_bmi['age_group_id']==10,25,np.where(ihd_bmi['age_group_id']==11,30,np.where(ihd_bmi['age_group_id']==12,35,np.where(ihd_bmi['age_group_id']==13,40,np.where(ihd_bmi['age_group_id']==14,45,np.where(ihd_bmi['age_group_id']==15,50,np.where(ihd_bmi['age_group_id']==16,55,np.where(ihd_bmi['age_group_id']==17,60,np.where(ihd_bmi['age_group_id']==18,65,np.where(ihd_bmi['age_group_id']==19,70,np.where(ihd_bmi['age_group_id']==20,75,np.where(ihd_bmi['age_group_id']==30,80,np.where(ihd_bmi['age_group_id']==31,85,np.where(ihd_bmi['age_group_id']==32,90,0))))))))))))))
ihd_bmi['sex'] = np.where(ihd_bmi['sex_id']==1,'Male','Female')
ihd_bmi = ihd_bmi.rename(columns={"draw_0": "ihd_bmi_rr"}).reset_index()
ihd_bmi = ihd_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_bmi.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,ihd_bmi_rr,age_start,sex
0,0,0,14,2,1.591258,45,Female
1,1,1,17,2,1.379902,60,Female
2,2,2,12,2,1.685112,35,Female
3,3,3,13,2,1.665879,40,Female
4,4,4,20,2,1.291149,75,Female


In [3]:
stroke_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                    age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                    sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_bmi = stroke_bmi.loc[stroke_bmi.cause_id==495].reset_index()
stroke_bmi = stroke_bmi[['age_group_id','sex_id','draw_0']]
stroke_bmi['age_start'] = np.where(stroke_bmi['age_group_id']==10,25,np.where(stroke_bmi['age_group_id']==11,30,np.where(stroke_bmi['age_group_id']==12,35,np.where(stroke_bmi['age_group_id']==13,40,np.where(stroke_bmi['age_group_id']==14,45,np.where(stroke_bmi['age_group_id']==15,50,np.where(stroke_bmi['age_group_id']==16,55,np.where(stroke_bmi['age_group_id']==17,60,np.where(stroke_bmi['age_group_id']==18,65,np.where(stroke_bmi['age_group_id']==19,70,np.where(stroke_bmi['age_group_id']==20,75,np.where(stroke_bmi['age_group_id']==30,80,np.where(stroke_bmi['age_group_id']==31,85,np.where(stroke_bmi['age_group_id']==32,90,0))))))))))))))
stroke_bmi['sex'] = np.where(stroke_bmi['sex_id']==1,'Male','Female')
stroke_bmi = stroke_bmi.rename(columns={"draw_0": "stroke_bmi_rr"}).reset_index()
stroke_bmi = stroke_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_bmi.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,stroke_bmi_rr,age_start,sex
0,0,0,14,2,1.809969,45,Female
1,1,1,15,2,1.727334,50,Female
2,2,2,13,2,1.738278,40,Female
3,3,3,18,2,1.344813,65,Female
4,4,4,10,2,2.405075,25,Female


In [4]:
ihd_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_fpg = ihd_fpg.loc[ihd_fpg.cause_id==493].reset_index()
ihd_fpg = ihd_fpg[['age_group_id','sex_id','draw_0']]
ihd_fpg['age_start'] = np.where(ihd_fpg['age_group_id']==10,25,np.where(ihd_fpg['age_group_id']==11,30,np.where(ihd_fpg['age_group_id']==12,35,np.where(ihd_fpg['age_group_id']==13,40,np.where(ihd_fpg['age_group_id']==14,45,np.where(ihd_fpg['age_group_id']==15,50,np.where(ihd_fpg['age_group_id']==16,55,np.where(ihd_fpg['age_group_id']==17,60,np.where(ihd_fpg['age_group_id']==18,65,np.where(ihd_fpg['age_group_id']==19,70,np.where(ihd_fpg['age_group_id']==20,75,np.where(ihd_fpg['age_group_id']==30,80,np.where(ihd_fpg['age_group_id']==31,85,np.where(ihd_fpg['age_group_id']==32,90,0))))))))))))))
ihd_fpg['sex'] = np.where(ihd_fpg['sex_id']==1,'Male','Female')
ihd_fpg = ihd_fpg.rename(columns={"draw_0": "ihd_fpg_rr"}).reset_index()
ihd_fpg = ihd_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_fpg.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,ihd_fpg_rr,age_start,sex
0,0,0,10,2,1.9325,25,Female
1,1,1,11,2,1.6915,30,Female
2,2,2,12,2,1.4505,35,Female
3,3,3,13,2,1.3135,40,Female
4,4,4,14,2,1.2805,45,Female


In [5]:
stroke_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_fpg = stroke_fpg.loc[stroke_fpg.cause_id==495].reset_index()
stroke_fpg = stroke_fpg[['age_group_id','sex_id','draw_0']]
stroke_fpg['age_start'] = np.where(stroke_fpg['age_group_id']==10,25,np.where(stroke_fpg['age_group_id']==11,30,np.where(stroke_fpg['age_group_id']==12,35,np.where(stroke_fpg['age_group_id']==13,40,np.where(stroke_fpg['age_group_id']==14,45,np.where(stroke_fpg['age_group_id']==15,50,np.where(stroke_fpg['age_group_id']==16,55,np.where(stroke_fpg['age_group_id']==17,60,np.where(stroke_fpg['age_group_id']==18,65,np.where(stroke_fpg['age_group_id']==19,70,np.where(stroke_fpg['age_group_id']==20,75,np.where(stroke_fpg['age_group_id']==30,80,np.where(stroke_fpg['age_group_id']==31,85,np.where(stroke_fpg['age_group_id']==32,90,0))))))))))))))
stroke_fpg['sex'] = np.where(stroke_fpg['sex_id']==1,'Male','Female')
stroke_fpg = stroke_fpg.rename(columns={"draw_0": "stroke_fpg_rr"}).reset_index()
stroke_fpg = stroke_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_fpg.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,stroke_fpg_rr,age_start,sex
0,0,0,10,2,1.5236,25,Female
1,1,1,11,2,1.3974,30,Female
2,2,2,12,2,1.2711,35,Female
3,3,3,13,2,1.2058,40,Female
4,4,4,14,2,1.2012,45,Female


In [6]:
ihd_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_sbp = ihd_sbp.loc[ihd_sbp.cause_id==493].reset_index()
ihd_sbp = ihd_sbp[['age_group_id','sex_id','draw_0']]
ihd_sbp['age_start'] = np.where(ihd_sbp['age_group_id']==10,25,np.where(ihd_sbp['age_group_id']==11,30,np.where(ihd_sbp['age_group_id']==12,35,np.where(ihd_sbp['age_group_id']==13,40,np.where(ihd_sbp['age_group_id']==14,45,np.where(ihd_sbp['age_group_id']==15,50,np.where(ihd_sbp['age_group_id']==16,55,np.where(ihd_sbp['age_group_id']==17,60,np.where(ihd_sbp['age_group_id']==18,65,np.where(ihd_sbp['age_group_id']==19,70,np.where(ihd_sbp['age_group_id']==20,75,np.where(ihd_sbp['age_group_id']==30,80,np.where(ihd_sbp['age_group_id']==31,85,np.where(ihd_sbp['age_group_id']==32,90,0))))))))))))))
ihd_sbp['sex'] = np.where(ihd_sbp['sex_id']==1,'Male','Female')
ihd_sbp = ihd_sbp.rename(columns={"draw_0": "ihd_sbp_rr"}).reset_index()
ihd_sbp = ihd_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_sbp.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,ihd_sbp_rr,age_start,sex
0,0,0,10,1,2.0317,25,Male
1,1,1,11,1,1.8558,30,Male
2,2,2,12,1,1.6799,35,Male
3,3,3,13,1,1.5729,40,Male
4,4,4,14,1,1.5348,45,Male


In [7]:
stroke_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_sbp = stroke_sbp.loc[stroke_sbp.cause_id==495].reset_index()
stroke_sbp = stroke_sbp[['age_group_id','sex_id','draw_0']]
stroke_sbp['age_start'] = np.where(stroke_sbp['age_group_id']==10,25,np.where(stroke_sbp['age_group_id']==11,30,np.where(stroke_sbp['age_group_id']==12,35,np.where(stroke_sbp['age_group_id']==13,40,np.where(stroke_sbp['age_group_id']==14,45,np.where(stroke_sbp['age_group_id']==15,50,np.where(stroke_sbp['age_group_id']==16,55,np.where(stroke_sbp['age_group_id']==17,60,np.where(stroke_sbp['age_group_id']==18,65,np.where(stroke_sbp['age_group_id']==19,70,np.where(stroke_sbp['age_group_id']==20,75,np.where(stroke_sbp['age_group_id']==30,80,np.where(stroke_sbp['age_group_id']==31,85,np.where(stroke_sbp['age_group_id']==32,90,0))))))))))))))
stroke_sbp['sex'] = np.where(stroke_sbp['sex_id']==1,'Male','Female')
stroke_sbp = stroke_sbp.rename(columns={"draw_0": "stroke_sbp_rr"}).reset_index()
stroke_sbp = stroke_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_sbp.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,stroke_sbp_rr,age_start,sex
0,0,0,10,1,1.7644,25,Male
1,1,1,11,1,1.7026,30,Male
2,2,2,12,1,1.6409,35,Male
3,3,3,13,1,1.5878,40,Male
4,4,4,14,1,1.5435,45,Male


In [8]:
ihd_ldl = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=367, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_ldl = ihd_ldl.loc[ihd_ldl.cause_id==493].reset_index()
ihd_ldl = ihd_ldl[['age_group_id','sex_id','draw_0']]
ihd_ldl['age_start'] = np.where(ihd_ldl['age_group_id']==10,25,np.where(ihd_ldl['age_group_id']==11,30,np.where(ihd_ldl['age_group_id']==12,35,np.where(ihd_ldl['age_group_id']==13,40,np.where(ihd_ldl['age_group_id']==14,45,np.where(ihd_ldl['age_group_id']==15,50,np.where(ihd_ldl['age_group_id']==16,55,np.where(ihd_ldl['age_group_id']==17,60,np.where(ihd_ldl['age_group_id']==18,65,np.where(ihd_ldl['age_group_id']==19,70,np.where(ihd_ldl['age_group_id']==20,75,np.where(ihd_ldl['age_group_id']==30,80,np.where(ihd_ldl['age_group_id']==31,85,np.where(ihd_ldl['age_group_id']==32,90,0))))))))))))))
ihd_ldl['sex'] = np.where(ihd_ldl['sex_id']==1,'Male','Female')
ihd_ldl = ihd_ldl.rename(columns={"draw_0": "ihd_ldl_rr"}).reset_index()
ihd_ldl = ihd_ldl.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_ldl.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,ihd_ldl_rr,age_start,sex
0,0,0,10,1,2.0053,25,Male
1,1,1,11,1,2.0267,30,Male
2,2,2,12,1,2.0483,35,Male
3,3,3,13,1,1.9898,40,Male
4,4,4,14,1,1.8515,45,Male


In [9]:
stroke_ldl = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=367, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_ldl = stroke_ldl.loc[stroke_ldl.cause_id==495].reset_index()
stroke_ldl = stroke_ldl[['age_group_id','sex_id','draw_0']]
stroke_ldl['age_start'] = np.where(stroke_ldl['age_group_id']==10,25,np.where(stroke_ldl['age_group_id']==11,30,np.where(stroke_ldl['age_group_id']==12,35,np.where(stroke_ldl['age_group_id']==13,40,np.where(stroke_ldl['age_group_id']==14,45,np.where(stroke_ldl['age_group_id']==15,50,np.where(stroke_ldl['age_group_id']==16,55,np.where(stroke_ldl['age_group_id']==17,60,np.where(stroke_ldl['age_group_id']==18,65,np.where(stroke_ldl['age_group_id']==19,70,np.where(stroke_ldl['age_group_id']==20,75,np.where(stroke_ldl['age_group_id']==30,80,np.where(stroke_ldl['age_group_id']==31,85,np.where(stroke_ldl['age_group_id']==32,90,0))))))))))))))
stroke_ldl['sex'] = np.where(stroke_ldl['sex_id']==1,'Male','Female')
stroke_ldl = stroke_ldl.rename(columns={"draw_0": "stroke_ldl_rr"}).reset_index()
stroke_ldl = stroke_ldl.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_ldl.head()

/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id
/ihme/homes/lutzes/.conda/envs/cvd/lib/python3.8/site-packages/get_draws/base/utils.py:30: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df.groupby([group_cols]):


,level_0,index,age_group_id,sex_id,stroke_ldl_rr,age_start,sex
0,0,0,10,1,1.4165,25,Male
1,1,1,11,1,1.4715,30,Male
2,2,2,12,1,1.5265,35,Male
3,3,3,13,1,1.5155,40,Male
4,4,4,14,1,1.4385,45,Male


### Now we can calculate the deltas
Note that we are finding the same value (deltas) based on two causes (IHD and stroke). Ideally, these should be identical. We find that they are very similar, making me more confident in these calculations.

In [10]:
sbp_ihd_mf = 0.31
ldl_ihd_mf = 0.10
fpg_ihd_mf = 0.15

sbp_stroke_mf = 0.65
ldl_stroke_mf = 0.03
fpg_stroke_mf = 0.22

In [33]:
bmi_delta = ihd_bmi.merge(ihd_fpg[['ihd_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(
    ihd_sbp[['ihd_sbp_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_bmi[['stroke_bmi_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_fpg[['stroke_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_ldl[['stroke_ldl_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_sbp[['stroke_sbp_rr','age_start','sex']], on=['age_start','sex'])

bmi_delta['fpg_delta_1'] = np.log((fpg_ihd_mf*(bmi_delta['ihd_bmi_rr']-1))+1)/np.log(bmi_delta['ihd_fpg_rr'])
bmi_delta['fpg_delta_2'] = np.log((fpg_stroke_mf*(bmi_delta['stroke_bmi_rr']-1))+1)/np.log(bmi_delta['stroke_fpg_rr'])
bmi_delta['sbp_delta_1'] = np.log((sbp_ihd_mf*(bmi_delta['ihd_bmi_rr']-1))+1)/np.log(bmi_delta['ihd_sbp_rr'])
bmi_delta['sbp_delta_2'] = np.log((sbp_stroke_mf*(bmi_delta['stroke_bmi_rr']-1))+1)/np.log(bmi_delta['stroke_sbp_rr'])

bmi_delta['delta_diff_fpg'] = bmi_delta['fpg_delta_1'] - bmi_delta['fpg_delta_2']
bmi_delta['delta_diff_sbp'] = bmi_delta['sbp_delta_1'] - bmi_delta['sbp_delta_2']

bmi_delta = bmi_delta[['ihd_bmi_rr','ihd_fpg_rr','ihd_sbp_rr','stroke_bmi_rr','stroke_sbp_rr','stroke_fpg_rr','age_start', 'sex','fpg_delta_1','sbp_delta_1', 'fpg_delta_2','sbp_delta_2','delta_diff_fpg','delta_diff_sbp']]
bmi_delta

,ihd_bmi_rr,ihd_fpg_rr,ihd_sbp_rr,stroke_bmi_rr,stroke_sbp_rr,stroke_fpg_rr,age_start,sex,fpg_delta_1,sbp_delta_1,fpg_delta_2,sbp_delta_2,delta_diff_fpg,delta_diff_sbp
0,1.591258,1.2805,1.5348,1.809969,1.5435,1.2012,45,Female,0.343675,0.392854,0.894507,0.974454,-0.550832,-0.581600
1,1.379902,1.1815,1.4203,1.497946,1.4105,1.1877,60,Female,0.332289,0.317314,0.604311,0.815261,-0.272022,-0.497947
2,1.685112,1.4505,1.6799,1.919950,1.6409,1.2711,35,Female,0.263028,0.371268,0.768336,0.946466,-0.505308,-0.575199
3,1.665879,1.3135,1.5729,1.738278,1.5878,1.2058,40,Female,0.349118,0.414331,0.804223,0.847760,-0.455106,-0.433429
4,1.291149,1.1897,1.2947,1.273751,1.2820,1.1660,75,Female,0.246087,0.334571,0.380790,0.659225,-0.134703,-0.324654
5,1.296208,1.1512,1.3402,1.271957,1.3233,1.1760,70,Female,0.308742,0.300015,0.358435,0.581073,-0.049693,-0.281058
6,2.094858,1.6915,1.8558,2.220086,1.7026,1.3974,30,Female,0.289297,0.472616,0.710585,1.097274,-0.421288,-0.624657
7,3.510608,1.9325,2.0317,2.405075,1.7644,1.5236,25,Female,0.485129,0.812066,0.639677,1.142685,-0.154547,-0.330619
8,1.382845,1.2475,1.4966,1.727334,1.4992,1.1968,50,Female,0.252501,0.278156,0.826221,0.956070,-0.573720,-0.677915
9,1.425974,1.2145,1.4584,1.556205,1.4548,1.1923,55,Female,0.318720,0.328701,0.656330,0.823253,-0.337610,-0.494552


In [12]:
print(bmi_delta['delta_diff_sbp'].mean())
print(bmi_delta['delta_diff_sbp'].min())
print(bmi_delta['delta_diff_fpg'].mean())
print(bmi_delta['delta_diff_fpg'].min())

-0.37932985661960256
-0.6779145317209673
-0.25589677366874863
-0.5737199459174113


The deltas here are quite different now... which is a bit concerning to our approach. I think using the IHD delta values makes the most sense as IHD is more likely to be similar to HF. 

### The RR for SBP to HF is categorical, we need continuous
The literature value for SBP RR to HF are in categories. To account for this, we found a continuous RR that approximates the same values. The mean_SBP are based on exposure values form GBD.

In [13]:
SBP_RR = pd.DataFrame(np.array([['<120',1,110.6],['120-129',1.27,124.4],['130-139',1.5,134.2],['140+',1.76,150.7]]),
                   columns=['SBP_Group', 'RR_original', 'Mean_SBP'])
SBP_RR['Mean_SBP'] = pd.to_numeric(SBP_RR['Mean_SBP'])
SBP_RR['Change_in_SBP'] = SBP_RR['Mean_SBP'] - 110.6
cont_RR = 1.016
SBP_RR['New_RR'] = np.power(cont_RR,SBP_RR['Change_in_SBP'])
SBP_RR

,SBP_Group,RR_original,Mean_SBP,Change_in_SBP,New_RR
0,<120,1,110.6,0.0,1.000000
1,120-129,1.27,124.4,13.8,1.244896
2,130-139,1.5,134.2,23.6,1.454426
3,140+,1.76,150.7,40.1,1.889895


### Lastly, we find the Mediation Factors

In [14]:
rr_sbp = 1.016
rr_bmi = 1.14

In [34]:
bmi_delta['sbp_delta_1'] = bmi_delta.sbp_delta_1 * 2 
bmi_delta = bmi_delta[['age_start', 'sex', 'sbp_delta_1']]
bmi_delta#.head()

,age_start,sex,sbp_delta_1
0,45,Female,0.785708
1,60,Female,0.634628
2,35,Female,0.742535
3,40,Female,0.828662
4,75,Female,0.669142
5,70,Female,0.600031
6,30,Female,0.945233
7,25,Female,1.624132
8,50,Female,0.556311
9,55,Female,0.657401


In [32]:
HF_MF=pd.DataFrame(bmi_delta)
HF_MF.to_csv('/ihme/homes/lutzes/heart_failure_MF.csv')

### Testing based on the intervention if we need FPG
This is bit tough/estimated since right now I am extracting data from a figure. I hope to find better data at some point. I am choosing to assess this at 1 year since that is when there is the greatest change in both BMI and FPG.

In [17]:
#int_bmi = 2.5
#int_fpg = 0.3 mmol/L --> 5 mg/dL

For BMI will account for a 1/2 unit change in FPG, the intervention saw a ~5 unit change, we should add mediation

## New Testing Based on Zeb's Calculations

In [18]:
bmi_ihd = ihd_bmi.loc[(ihd_bmi.age_start==70) & (ihd_bmi.sex == 'Female')].ihd_bmi_rr.values[0]
sbp_ihd = ihd_sbp.loc[(ihd_sbp.age_start==70) & (ihd_sbp.sex == 'Female')].ihd_sbp_rr.values[0]
ldl_ihd = ihd_ldl.loc[(ihd_ldl.age_start==70) & (ihd_ldl.sex == 'Female')].ihd_ldl_rr.values[0]
fpg_ihd = ihd_fpg.loc[(ihd_fpg.age_start==70) & (ihd_fpg.sex == 'Female')].ihd_fpg_rr.values[0]

sbp_mf = 0.31
ldl_mf = 0.10
fpg_mf = 0.15
print(bmi_ihd,sbp_ihd,ldl_ihd,fpg_ihd)

1.296208 1.3401999 1.2347 1.1512001


In [19]:
## Alternative way 
## Δ_(r→m)=(log⁡(MF∗(RR_(R→O,total)−1)+1))/(log⁡(RR_(M→O)))
delta_sbp = np.log((sbp_mf*(bmi_ihd-1))+1)/np.log(sbp_ihd)
delta_ldl = np.log(ldl_mf*(bmi_ihd-1)+1)/np.log(ldl_ihd)
delta_fpg = np.log(fpg_mf*(bmi_ihd-1)+1)/np.log(fpg_ihd)
print(delta_sbp,delta_ldl,delta_fpg)

0.3000153583753123 0.13845682533912737 0.308742177413229


In [20]:
## Method used today
## RR adj = 1+ (RR unad -1) * (1-MF_SBP) * (1-MF_LDL) * (1-MF_FPG)
RR_adj_1 = 1+(bmi_ihd -1)*(1-sbp_mf)*(1-ldl_mf)*(1-fpg_mf)
RR_adj_1

1.1563533928

In [21]:
RR_adj_1 * ((pow(sbp_ihd, delta_sbp))*(pow(ldl_ihd, delta_ldl))*(pow(fpg_ihd, delta_fpg)))

1.3576897859933077

In [22]:
## RR_adj=(BMI_IHD)/(∏_m▒(RR_(m→o) )^(Δ_(r→m)))
RR_adj_2=(bmi_ihd)/((pow(sbp_ihd, delta_sbp))*(pow(ldl_ihd, delta_ldl))*(pow(fpg_ihd, delta_fpg)))
RR_adj_2

1.1039889480187124

In [23]:
RR_adj_2 * ((pow(sbp_ihd, delta_sbp))*(pow(ldl_ihd, delta_ldl))*(pow(fpg_ihd, delta_fpg)))

1.296208

### Also for Stroke

In [24]:
bmi_stroke = stroke_bmi.loc[(stroke_bmi.age_start==70) & (stroke_bmi.sex == 'Female')].stroke_bmi_rr.values[0]
sbp_stroke = stroke_sbp.loc[(stroke_sbp.age_start==70) & (stroke_sbp.sex == 'Female')].stroke_sbp_rr.values[0]
ldl_stroke = stroke_ldl.loc[(stroke_ldl.age_start==70) & (stroke_ldl.sex == 'Female')].stroke_ldl_rr.values[0]
fpg_stroke = stroke_fpg.loc[(stroke_fpg.age_start==70) & (stroke_fpg.sex == 'Female')].stroke_fpg_rr.values[0]

sbp_mf = 0.65
ldl_mf = 0.03
fpg_mf = 0.22
print(bmi_stroke,sbp_ihd,ldl_ihd,fpg_ihd)

1.271957 1.3401999 1.2347 1.1512001


In [25]:
## Alternative way 
## Δ_(r→m)=(log⁡(MF∗(RR_(R→O,total)−1)+1))/(log⁡(RR_(M→O)))
delta_sbp = np.log((sbp_mf*(bmi_stroke-1))+1)/np.log(sbp_stroke)
delta_ldl = np.log(ldl_mf*(bmi_stroke-1)+1)/np.log(ldl_stroke)
delta_fpg = np.log(fpg_mf*(bmi_stroke-1)+1)/np.log(fpg_stroke)
print(delta_sbp,delta_ldl,delta_fpg)

0.5810728694359281 0.08123462688345602 0.3584347385074138


In [26]:
## Method used today
## RR adj = 1+ (RR unad -1) * (1-MF_SBP) * (1-MF_LDL) * (1-MF_FPG)
RR_adj_1 = 1+(bmi_stroke -1)*(1-sbp_mf)*(1-ldl_mf)*(1-fpg_mf)
RR_adj_1

1.07201693317

In [27]:
RR_adj_1 * ((pow(sbp_stroke, delta_sbp))*(pow(ldl_stroke, delta_ldl))*(pow(fpg_stroke, delta_fpg)))

1.347905131294933

In [28]:
## RR_adj=(BMI_IHD)/(∏_m▒(RR_(m→o) )^(Δ_(r→m)))
RR_adj_2=(bmi_stroke)/((pow(sbp_stroke, delta_sbp))*(pow(ldl_stroke, delta_ldl))*(pow(fpg_stroke, delta_fpg)))
RR_adj_2

1.0116138076825494

In [29]:
RR_adj_2 * ((pow(sbp_stroke, delta_sbp))*(pow(ldl_stroke, delta_ldl))*(pow(fpg_stroke, delta_fpg)))

1.271957